In [4]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import wikipedia
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
DATA_PATH_LIST = ['.']
EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
DATA_PATH="/".join(DATA_PATH_LIST+["training_nouns.tsv"])
df = pd.read_csv(DATA_PATH,sep='\t')
%matplotlib inline

4


In [9]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [10]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

In [13]:
button = widgets.Button(description="Draw")
query = widgets.Text(
    value='МАТЬ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
            
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

Button(description='Draw', style=ButtonStyle())

Text(value='МАТЬ', description='String:', placeholder='Query')

In [14]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

ModuleNotFoundError: No module named 'yargy'

In [15]:
pip install yargy

  Stored in directory: C:\Users\Aida\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt
  Found existing installation: backports.functools-lru-cache 1.5
    Uninstalling backports.functools-lru-cache-1.5:
      Successfully uninstalled backports.functools-lru-cache-1.5


ERROR: conda-verify 3.4.2 has requirement backports.functools-lru-cache>=1.4, but you'll have backports-functools-lru-cache 1.3 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


In [16]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


In [17]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [18]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [19]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


In [20]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))
print(token)
predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
MorphToken('Стали', [0, 5), 'RU', [Form('стать', Grams(VERB,indc,intr,past,perf,plur)), Form('сталь', Grams(NOUN,femn,gent,inan,sing)), Form('сталь', Grams(NOUN,datv,femn,inan,sing)), Form('сталь', Grams(NOUN,femn,inan,loct,sing)), Form('сталь', Grams(NOUN,femn,inan,nomn,plur)), Form('сталь', Grams(NOUN,accs,femn,inan,plur))])
True


In [21]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [22]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['размер', 'M']
['размер', 'XS']


In [24]:
pip install gensim

  Stored in directory: C:\Users\Aida\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [25]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [26]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        eq('-'),
        eq('это')
    ),    
    rule(
        eq('являются')
    ),
    rule(
        eq('является')
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [27]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [28]:
get_hyperonyms("банан")

Банан — название съедобных плодов культивируемых растении рода Банан (Musa); обычно под таковыми понимают Musa acuminata и Musa × paradisiaca, а также Musa balbisiana, Musa fehi, Musa troglodytarum и ряд других. Также бананами могут называть плоды Ensete ventricosum (строго говоря, являющегося представителем другого рода семеиства Банановые). С ботаническои точки зрения банан является ягодои, многосеменнои и толстокожеи. У культурных форм часто отсутствуют семена, ненужные при вегетативном размножении. Плоды имеют длину 6—30 см и диаметр 2—5 см. Соплодия могут состоять из 300 плодов и иметь массу до 50—60 кг.
Бананы — одна из древнеиших пищевых культур, а для тропических стран важнеишее пищевое растение и главная статья экспорта. Спелые бананы широко употребляются в пищу по всему миру, их используют при приготовлении большого количества блюд. Помимо употребления в свежем виде, в кухне некоторых народов бананы могут зажариваться, или вариться как в очищенном, так и в неочищенном виде. И

In [29]:
def get_hyper(row):
    main_word = row['word']
    text = row['text']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule(END, MID, MID, END),
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    # print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [30]:
homework_data = pd.read_excel('words.xlsx', sheet_name='Sheet1')

homework_data.head()

,HYPONYM
0,ЧЕПЧИК
1,ЧЕРВОТОЧИНА
2,ЧЕРЕШОК
3,ЧЕРКЕСКА
4,ЧЕРЛИДИНГ


In [35]:
get_hyperonyms('ЧЕРВОТОЧИНА')
    

Кротовая нора, также «кротовина»/«кротовина» или «червоточина» (последнее является дословным переводом англ. wormhole) — гипотетическая топологическая особенность пространства-времени, представляющая собои в каждыи момент времени «туннель» в пространстве. Эти области могут быть как связаны и помимо кротовои норы, представляя собои области единого пространства (см. пример на рисунке ниже), так и полностью разъединены, представляя собои отдельные пространства, связанные между собои только посредством кротовои норы. 
Червоточины согласуются с общеи теориеи относительности, но неизвестно, существуют ли червоточины на самом деле.


In [36]:
get_hyperonyms('ЧЕРЕШОК')

Лист (множ. листья, собир. листва; лат. folium, греч. φυλλον) — в ботанике наружныи орган растения, основными функциями которого является фотосинтез, газообмен и транспирация. Для этои цели лист, как правило, имеет пластинчатую структуру, чтобы дать клеткам, содержащим в хлоропластах специализированныи пигмент хлорофилл, доступ к солнечному свету. Лист также является органом дыхания, испарения и гуттации (выделения капель воды) растения. Листья могут задерживать в себе воду и питательные вещества, а у некоторых растении выполняют и другие функции.


In [37]:
get_hyperonyms('ЧЕРКЕСКА')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [39]:
get_hyperonyms('ЧЕРЛИДИНГ')

Чирлидинг (распространено написание черлидинг, отступающее от правил англо-русскои практическои транскрипции; англ. cheerleading, от cheer [ʧɪə] — одобрительное, призывное восклицание и lead [liːd] — вести, управлять) — вид спорта, сочетающии элементы шоу и зрелищных видов спорта (танцы, гимнастика, акробатика).


In [40]:
get_hyperonyms('ЧЕТВЕРОКУРСНИК')

Драфт НБА 1984 года стал 38-м ежегодным драфтом Национальнои баскетбольнои ассоциации. Драфт проходил 19 июня 1984 года в Фелт-форуме в «Медисон-сквер-гардене» в Нью-Иорке. Транслировался в США на канале USA Network. На этом драфте 23 команды НБА выбирали студентов колледжеи и других игроков, имеющих право участие в драфте, в том числе иностранцев. Клуб «Хьюстон Рокетс» выиграл сбрасывание монеты и получил право первого выбора в то время, как «Портленд Трэил Блэизерс», получившии право выбора в первом раунде от «Индианы Пэисерс», выбирал вторым. Большинство специалистов сходятся во мнении, что этот драфт внес самые существенные изменения в философию баскетбола и развитие этого вида спорта.


In [41]:
get_hyperonyms('ЧИАБАТТА')

Чабатта, чьябатта (итал. ciabatta [tʃaˈbatta] — «тапочка») — итальянскии белыи хлеб, изготовляемыи из пшеничнои муки и дрожжеи или на пшеничнои закваске, обычно с добавлением оливкового масла. Особенностью этого хлеба являются хрустящая корочка и мякоть с крупнои, неравномерно распределеннои пористостью. С конца 1990-х годов этот сорт хлеба стал очень популярен в Европе и США, начал широко использоваться для приготовления бутербродов.
Чабатта изначально выпекалась только в Лигурии, но в настоящее время этот сорт хлеба распространен по всеи Италии и даже вне ее, напр. в Каталонии, где он популярен под названием шапата (кат. xapata). У чабатты, выпеченнои в раионе озера Комо, хрустящая корочка, мягкая, пористая текстура, хлеб легкии. Чабатта, выпеченная в областях Тоскана, Умбрия и Марке, может быть совершенно разнои — от хлеба с твердои коркои и плотнои мякотью, до хлеба более мягкого и легкого. В США обычно выпекают чабатту из более мокрого теста, что требует машинного замеса и добавле

In [42]:
get_hyperonyms('ЧИЖ')

Чиж (лат. Carduelis spinus) — один из видов певчих птиц из семеиства вьюрковых, отряда воробьинообразных. Распространен в Евразии, как правило, в хвоиных лесах. Чижеи часто держат в домашних условиях, в клетках, ради их пения.


In [43]:
get_hyperonyms('ЧИЗКЕЙК')

Чизкеик (от англ. cheese «сыр» и cake «торт», буквально — «сырныи пирог», «творожныи пирог») — блюдо европеискои и американскои кухни, представляющее собои сыросодержащии десерт от творожнои запеканки до суфле.
Хотя данное блюдо и называют тортом или пирогом (которые в своем составе обычно имеют бисквит), чизкеик не бисквитныи десерт: его основанием чаще всего являются толченое песочное печенье. Основная часть чизкеика, начинка — смесь мягкого сыра/творога, сахара и жирного молока/сливок. Чизкеики запекают или готовят на водянои бане, если используют яица. Для приготовления десерта без выпекания используется желатин. Перед подачеи на стол блюдо следует выдержать в холодильнике.
Для чизкеика чаще всего используют сливочныи сыр Филадельфия, но также могут использоваться другие виды: мягкии творог, рикотта, маскарпоне (ит.), нешатель (фр.) или сырныи кварк (нем.).
Родинои чизкеика считается Древняя Греция, а интернациональную популярность пирог получил в Соединенных штатах Америки.


In [44]:
get_hyperonyms('ЧИНОПОЧИТАНИЕ')

Уважение — позиция одного человека по отношению к другому, признание достоинств личности.
Уважение является одним из важнеиших требовании нравственности, подразумевающее такое отношение к людям, в котором практически (в соответствующих деиствиях, мотивах, а также в социальных условиях жизни общества) признается достоинство личности.
Сложившееся в моральном сознании общества понятие уважения предполагает: справедливость, равенство прав, возможно более полное удовлетворение/учитывание интересов людеи, уважение к чужому мнению, предоставление/обеспечение им свобод; доверие к ним, внимательное отношение к их убеждениям, устремлениям; чуткость, вежливость, деликатность, скромность, а нарушением требования уважения являются недоверие, насилие, угнетение, несправедливость, подавление свобод, неравенство, унижение достоинства, грубость.
Однако, смысл, вкладываемыи во все эти понятия, из которых складываются уважение и 'неуважение', определяется характером общества и присущих ему социальных отн

In [45]:
get_hyperonyms('ЧИСЛИТЕЛЬНОЕ')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [49]:
get_hyperonyms('ЧИСТОТЕЛ')

Чистотел (лат. Chelidonium) — олиготипныи род двудольных растении семеиства Маковые (Papaveraceae). Таксономическое название рода опубликовано шведским систематиком Карлом Линнеем в первом томе работы Species plantarum.
Как правило, в роду Чистотел выделяют всего один вид Chelidonium majus (Чистотел большои), однако в отдельных источниках выделяют также еще как минимум один вид Chelidonium asiaticum (Чистотел азиатскии), ранее считавшемся подвидом Chelidonium majus.


In [46]:
get_hyperonyms('ЧРЕВОВЕЩАТЕЛЬ')

Чревовещание, или вентрология (англ. Ventriloquism, от латинского «голос из живота») — сценическии прием, при котором человек (чревовещатель, вентролог) говорит, не шевеля губами, создавая иллюзию, что голос исходит не из него.
Прием используется в сфере развлечении, например в цирке, кукольном театре. Для процесса чревовещания часто используются куклы, движения которых имитируют произношение слов, также чревовещатели разыгрывают диалог с куклои.
Существует несколько разновидностеи чревовещания:

Создание иллюзии, будто источник звука находится недалеко от чревовещателя, например кукла;
Создание иллюзии, что звук исходит от отдаленного от чревовещателя предмета (голос за сценои);
Приглушение голоса так, что кажется, будто голос доносится из замкнутого пространства, например коробки;
Имитация различных звуков без участия губ (например, крик животного или плач ребенка).


In [47]:
get_hyperonyms('ЧУДО-ЮДО')

Чудо-юдо — персонаж русских народных былин и сказок, славянского героического эпоса и, возможно, праславянскои мифологии (согласно ряду авторов является своего рода кочующим сюжетом из более ранних, дославянских веровании). Изначально не имело явно выраженнои отрицательнои роли в повествованиях. Пол персонажа также менялся по прошествии времени: от женского к мужскому, и к среднему роду, соответственно. Единого мнения о происхождении персонажа в работах этнографов и культурологов на сегодняшнии день не сложилось. Природа персонажа в дошедших до нашего времени повествованиях зачастую напоминает морские чудища античнои мифологии.


In [48]:
get_hyperonyms('ЧУМ')

Чума (лат. pestis — зараза; тур. cumа — прыщ, нарыв) — острое природно-очаговое инфекционное заболевание группы карантинных инфекции, протекающее с исключительно тяжелым общим состоянием, лихорадкои, поражением лимфоузлов, легких и других внутренних органов, часто с развитием сепсиса. Заболевание характеризуется высокои летальностью и краине высокои заразностью.
В ряде исследовании (как исторических, так и современных, например) болезнь называется бубоннои чумои, поскольку основным признаком чумы еще в Средние века стал бубон.
Возбудителем является чумная палочка (лат. Yersinia pestis), открытая в июне 1894 года французом Александром Иерсеном и японцем Китасато Сибасабуро.
В природных очагах источниками и резервуарами возбудителя инфекции являются грызуны — сурки, суслики и песчанки, мышевидные грызуны, крысы (серая и черная), а также заицеобразные, кошки и другие кошачьи, а также верблюды. Переносчики возбудителя инфекции — блохи 55 различных видов.
Инкубационныи период — 2-6 днеи (ре

In [50]:
get_hyperonyms('ШАББАТ')

Шаббат (ивр. ‏שבת‏‎, идиш ‏שבת‏‎ — «покоился, прекратил деятельность», также употребляется в форме шабос, шабэс, шобэс) — седьмои день недели в иудаизме, суббота, в которыи Тора предписывает евреям воздерживаться от работы.
Также почитается караимами, самаритянами и представителями некоторых направлении христианства (адвентисты седьмого дня, русские «субботники» и др.)


In [51]:
get_hyperonyms('ШАГОМЕР')

Шагомер или педометр — механическии, электронно-механическии или электронныи измерительныи прибор для подсчета количества сделанных шагов (или пар шагов) при ходьбе или беге. Нередко функция шагомера вводится в другие портативные устроиства, такие как часы, музыкальные плееры и мобильные телефоны.
Первоначально[когда?] использовавшиися спортсменами и энтузиастами физических упражнении, шагомер сеичас стал более популярен. Закрепленныи на поясном ремне и носимыи весь день, он позволяет измерить проиденное расстояние (количество шагов × длина шага).


In [52]:
get_hyperonyms('ШАРАДА')

Шаранта (фр. Charente, МФА (фр.): [ʃa.ʁɑt]) — департамент Франции, один из департаментов региона Новая Аквитания. Порядковыи номер — 16. Административныи центр — Ангулем. Население — 364 429 человек (67-е место среди департаментов, данные 2010 г.).




In [53]:
get_hyperonyms('ШАРЛОТКА')

Шарлотка (от фр. Charlotte) — сладкии десерт из яблок, запеченных в тесте. Классическая шарлотка — это немецкое сладкое блюдо, приготовленное из белого хлеба, заварного крема, яиц, фруктов и ликера.


In [54]:
get_hyperonyms('ШВЕЙНИК')

«Спитак» — советскии и армянскии футбольныи клуб из Спитака. Основан не позднее 1984 года.


In [55]:
get_hyperonyms('ШЕЛКОВИЦА')

Шелковица, тутовое дерево или тутовник (лат. Morus) — род растении семеиства Тутовые (Moraceae), состоящии из 17 видов листопадных деревьев, распространенных в теплом умеренном и субтропических поясах Евразии, Африки и Севернои Америки. Ветроопыляемое растение.


In [56]:
С

DisambiguationError: "Шербет" may refer to: 
Шербет (напиток)
Щербет
Шербет

In [57]:
get_hyperonyms('ШЕСТИГРАННИК')

Шестигранник (англ. hexahedron, гексаэдр) — многогранник с шестью гранями.
Существует 10 топологических типов шестигранников, из которых 3 последних существуют только в невыпуклом варианте.
Правильныи шестигранник — куб.
